# レポート課題第8回


Googleドライブのマウントとフォルダの移動

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/08/
# 現在のフォルダの中身を表示
%ls

必要ライブラリをインポート。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`gokyouka.csv` を読み込む。  

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('gokyouka.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) に変換する。
X = csv_data.to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

主成分分析を用いて2次元に圧縮する。

In [ ]:
# データの標準化
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 分散共分散行列の計算
cov = np.cov(X_norm.T, bias=True)
# 固有値分解
eig_val, eig_vec =np.linalg.eig(cov)

# 固有値を大きい順に並び替える
order = np.argsort(eig_val)[::-1]
eig_val = eig_val[order]
eig_vec_pca = eig_vec[:,order]

# 第二主成分までの固有ベクトルを用いて次元圧縮
X_pca = np.dot(X_norm, eig_vec_pca[:,:2])

# 二次元データのプロット
plt.figure(figsize=(10,6))
plt.scatter(X_pca[:,0], X_pca[:,1], color='k')
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.show()

## K-means による3クラスのクラスタリング  
まず，k-meansによるクラスタリングを実行する。  
以下は k-means の関数の実装(08_01_kmeans.ipynb参照)。

In [ ]:
def euclidean_distance(x, y):
  '''
      ユークリッド距離を計算して出力する
      x, y: 距離を測りたい2個の多次元ベクトル
  '''
  dist = np.sum((x-y)**2)
  return dist

def k_means(centroid, data):
  '''
     k-means によるセントロイドの更新を1回行う。
     入力
     centroid: 現在のセントロイド
     data: データ
     出力
     new_centroid: 更新後のセントロイド
     assigned_class: 各データが振り分けられたクラス番号
  '''
  # クラス数を取得
  num_classes, num_dimensions = np.shape(centroid)
  # データのサンプル数と次元数を取得
  num_samples, num_dimensions = np.shape(data)

  # 各データに対して割り当てるクラス番号
  assigned_class = np.zeros(num_samples)

  # 各データに対して，最もセントロイドが近いクラスを割り当てる
  for n in range(num_samples):
    # 各クラスのセントロイドとの距離を測る
    dists = np.zeros(num_classes)
    for k in range(num_classes):
      # n番目のデータとk番目のセントロイドの距離
      dists[k] = euclidean_distance(data[n,:], centroid[k,:])

    # セントロイドとの距離が最小のクラス番号を割り当てる
    assigned_class[n] = np.argmin(dists)
  
  # セントロイドの更新
  new_centroid = np.zeros((num_classes, num_dimensions))
  for k in range(num_classes):
    # k番目のクラスに該当するデータ data[assigned_class==k] の平均値を
    # k番目のクラスの新たなセントロイドとする
    new_centroid[k] = np.mean(data[assigned_class==k,:], axis=0)
  
  return new_centroid, assigned_class

分割するクラスの数 $K=3$ としてセントロイド初期値を設定し，k-meansを実行する。

In [ ]:
#クラス数
K = 3

#
# セントロイド初期値の設定(06_01_kmeans.ipynb参照)
#
# 乱数シードの設定(ここではシードを5としていますが，何でも良いです)
np.random.seed(5)
# 0～99 をランダムに並び替える
shuffle_index = np.random.permutation(np.arange(num_samples))
# 並び替えた数の先頭 K個を取り出して、セントロイドの初期値とする。
centroid_index = shuffle_index[:K]
initial_centroid = X_pca[centroid_index,:]


#
# K-means を実行する(06_01_kmeans.ipynb参照)
#

# 最大更新回数を100とする
max_iter = 100

centroid = initial_centroid

# ループの回数は max_iter とする
for n in range(max_iter):
  print('Iteration: ' + str(n+1))

  # k-meansによるセントロイドの更新を実施
  new_centroid, assigned_class = k_means(centroid, X_pca)

  # セントロイドが変わっていない，
  # つまり更新前後のセントロイドの移動距離が0の場合，ループを抜ける
  if np.sum((new_centroid - centroid)**2) == 0:
    print('centroids converged')
    break
  
  # セントロイドの更新
  centroid = new_centroid

結果をプロットする。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロット
  plt.scatter(X_pca[assigned_class==k,0], X_pca[assigned_class==k,1], color=c, label='class: '+str(k+1))
  # セントロイドを "x"印でプロット
  plt.scatter(centroid[k,0], centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

上記の結果では，成績が普通のクラス(文系)と成績が悪いクラスが緑色のクラス3でまとまってクラス化されていることから，この結果は直感と異なるクラスタリングがされていると言える。


## 混合正規分布によるクラスタリング
k-meansによるクラスタリング結果をさらによくするために，混合正規分布によるクラスタリングを行う。

まず，以下は多変量正規分布によるスコアを計算する関数 `normal_distribution` である。  
$f(x) = \frac{exp(-\frac{1}{2}(x-\mu)^{T}\Sigma^{-1}(x-\mu))}{\sqrt{(2\pi)^{D}|\Sigma|}}$  
$x$: スコアを計算したいデータサンプル($D$次元のベクトル)  
$\mu$: 正規分布の平均値ベクトル($D$次元ベクトル)  
$\Sigma$: 正規分布の分散共分散行列($D \times D$の行列)  
$\Sigma^{-1}$: 分散共分散行列の逆行列  
$|\Sigma|$: 分散共分散行列の行列式

In [ ]:
def normal_distribution(x, mean, cov):
  '''
      正規分布の確率密度関数の値を計算する
      x: 計算したいデータ
      mean: 平均値ベクトル
      cov: 分散共分散行列
  '''
  # 次元数
  D = np.size(x)
  # 分散共分散行列の逆行列
  inv_cov = np.linalg.inv(cov)
  # 分散共分散行列の行列式
  det_cov = np.linalg.det(cov)
  
  sub = x - mean
  # sub は1行k列のベクトルなので，これをk行1列のベクトルに変換する
  sub = np.reshape(sub, [-1,1])

  tmp = np.dot(sub.T, inv_cov)
  tmp = np.dot(tmp, sub)
  tmp = np.exp(-1.0 * tmp / 2)
  res = tmp / (np.sqrt((2*np.pi)**D * det_cov))

  return res[0,0]

## <font color=red>問題はここから</font>
<font color=red>**小問1: 以下の関数を補完し，各データの各クラス分布に対する帰属度$r_{n,k}$を計算する関数を実装せよ（穴埋め2か所）**</font>  
帰属度$r$はプログラム中ではratioと定義されています。  
重み付き正規分布関数値$s$はプログラム中ではscoresと定義されています。

In [ ]:
def update_ratio(X, weights, means, covs):
  '''
      X: 全データ
      weights: weights[k] はクラスkの重み
      means: KxDの配列。means[k] はクラスkの平均値ベクトル(次元数D)
      covs: KxDxDの三次元配列。covs[k] はクラスkの分散共分散行列(次元数DxD)
  '''
  # データ数Nと次元数Dを得る
  N, D = np.shape(X)
  # クラス数Mを得る
  K = np.size(weights)

  # 各データの，各クラス分布に対するスコア
  # score[n, k] は n番目のサンプルがクラスkの分布に対する重み付きスコアが格納される。
  scores = np.zeros((N, K))
  for n in range(N):
    for k in range(K):
      # スコアはクラスkの正規分布スコアに，クラスkの重みをかけたもの
      scores[n,k] = 
    
  # 各データの各クラス分布に対する帰属度
  # ratio[n, k] はn番目のサンプルがクラスkの分布に対する帰属度が格納される。
  ratio = np.zeros((N,K))
  for n in range(N):
    # 帰属度を全クラス総和が1になるよう正規化
    ratio[n] =
  
  return ratio

<font color=red>**小問2: 以下の関数を補完し，各クラスの重み$w_k$を計算する関数を実装せよ(穴埋め1か所)**</font>  
重み$w_k$はプログラム中ではweightsと定義している。

In [ ]:
def update_weights(ratio):
  '''
     ratio: NxKの配列。ratio[n,k]はn番目のサンプルがクラスkの分布に対する帰属度が格納される。
  '''
  # weights は ratio をサンプル数の方向へ平均することで得られる。
  weights = 
  return weights

<font color=red>**小問3: 以下の関数を補完し，各クラスの平均値ベクトル$\mu_{k}$を計算する関数を実装せよ(穴埋め3か所)**</font>  
平均値ベクトル$\mu_k$はプログラム中ではmeanと定義されている。

In [ ]:
def update_means(X, ratio):
  '''
      X: データ。NxDの配列。
      ratio: NxKの配列。ratio[n,k]はn番目のサンプルがクラスkの分布に対する帰属度が格納される。
  '''
  # データ数と次元数を得る
  N, D = np.shape(X)
  # クラス数を得る
  K = np.shape(ratio)[1]

  # means: MxDの配列。means[k] はクラスkの平均値ベクトル(次元数D)
  means = np.zeros((K, D))
  for k in range(K):
    # 分母の計算
    denom = 
    # 分子の計算
    numer = np.zeros(D)
    for n in range(N):
      numer += 
    # means[k,:] = 分子/分母
    means[k,:] = 
  
  return means

**以下の関数は各クラスの分散共分散行列$\Sigma_k$を計算する関数である。**  
分散共分散行列$\Sigma$はプログラム中ではcovsと定義されている。

In [ ]:
def update_covs(X, ratio, means):
  '''
      X: データ。NxDの配列。
      ratio: NxKの配列。ratio[n,k]はn番目のサンプルがクラスkの分布に対する帰属度が格納される。
      means: KxDの配列。means[k] はクラスkの平均値ベクトル(次元数D)
  '''
  # データ数と次元数を得る
  N, D = np.shape(X)
  # クラス数を得る
  K = np.shape(ratio)[1]
  
  # covs: MxDxDの三次元配列。covs[k] はクラスkの分散共分散行列(次元数DxD)
  covs = np.zeros((K, D, D))
  for k in range(K):
    # 分母の計算
    denom = np.sum(ratio[:,k])
    # 分子の計算
    numer = np.zeros((D,D))
    for n in range(N):
      sub = X[n] - means[k]
      # sub は1行k列のベクトルなので，これをk行1列のベクトルに変換する
      sub = np.reshape(sub, [-1,1])
      numer += ratio[n, k] * np.dot(sub, sub.T)
    # covs[k,:,:] = 分子/分母
    covs[k, :, :] = numer/denom
  
  return covs

<font color=red>**以下のセルは上記の関数の動作確認をするためのコードである**</font>  
実行すると以下のような出力が得られるはずである。  
デバッグの際に利用されたい。  

update_ratio  
[[0.99891715 0.00108285]  
 [0.96765775 0.03234225]  
 [0.29168584 0.70831416]]  
update_weights  
[0.75275358 0.24724642]  
update_means  
[[2.37364956 3.37364956]  
 [4.90695395 5.90695395]]  
update_covs  
[[[2.03331148 2.03331148]  
  [2.03331148 2.03331148]]  

 [[9.75604743 9.75604743]  
  [9.75604743 9.75604743]]]  


In [ ]:
Xtest = np.array([[1,2],[3,4],[5,6]])
wtest = np.array([0.4, 0.6])
mtest = np.array([[2, 3],[8,9]])
ctest = np.array([[[3,1],[1,3]],
                  [[5,2],[2,5]]])


ratio = update_ratio(Xtest, wtest, mtest, ctest)
print('update_ratio')
print(ratio)
print('update_weights')
print(update_weights(ratio))
print('update_means')
print(update_means(Xtest, ratio))
print('update_covs')
print(update_covs(Xtest, ratio, mtest))

<font color=red>**問題はここまで。動作確認がうまく行けば，以下も想定通り動くはずである。**</font>  
以下は，上記の小問で定義した関数を使って，混合正規分布の各パラメータを1回更新する関数である。  
また，各データについて，帰属度が最大となるクラスをargmax関数を使って取り出し，クラスの割り当て処理を行っている。

In [ ]:
def gmm_fit(X, weights, means, covs):
  # 帰属度を計算する
  ratio = update_ratio(X, weights, means, covs)

  # 重みを計算する。
  new_weights = update_weights(ratio)

  # 平均値ベクトルを計算する。
  new_means = update_means(X, ratio)

  # 分散共分散行列を計算する。
  new_covs = update_covs(X, ratio, new_means)

  # 帰属度が最も大きいクラスを，各データの属するクラスに割り当てる。
  assigned_class = np.argmax(ratio, axis=1)

  return new_weights, new_means, new_covs, assigned_class

k-meansの結果をもとに，means，covs，weightsの初期値を計算する。  
クラス毎に，k-meansによって割り当てられたデータを用いて平均と分散共分散行列を計算する。  
また割り当てられたデータ数を元にweightsも計算する。  

In [ ]:
weights = np.zeros(K)
means = np.zeros((K, 2))
covs = np.zeros((K, 2, 2))

for k in range(K):
  # assigned_classがkのデータを用いて平均/分散共分散行列を計算
  means[k] = np.mean(X_pca[assigned_class==k,:], axis=0)
  covs[k] = np.cov(X_pca[assigned_class==k,:].T)
  # assigned_classがkのデータ数/総データ数
  weights[k] = np.sum(assigned_class==k) / num_samples

混合正規分布によるクラスタリングを実行する。

In [ ]:
# 最大更新回数を100とする
max_iter = 100

# 各更新における平均値ベクトルおよびクラス振り分け結果の履歴を格納する変数
means_history = np.empty((0, K, 2), float)
assigned_class_gmm_history = np.empty((0, num_samples), int)

# ループの回数は max_iter とする
for n in range(max_iter):
  print('Iteration: %d' %(n+1) )

  # 混合正規分布の各パラメータの更新を実施
  new_weights, new_means, new_covs, assigned_class_gmm = gmm_fit(X_pca, weights, means, covs)

  # 更新結果を履歴に追加
  means_history = np.append(means_history, np.reshape(new_means,[-1,K,2]), axis=0)
  assigned_class_gmm_history = np.append(assigned_class_gmm_history, np.reshape(assigned_class_gmm,[-1,num_samples]), axis=0)

  # 平均値ベクトルが変わっていない，
  # つまり更新前後の平均値ベクトルの移動距離が小さい場合，ループを抜ける
  if np.sum((new_means-means)**2) < 1E-10:
    print('parameters converged')
    break
  
  # パラメータの更新
  weights = new_weights
  means = new_means
  covs = new_covs

混合正規分布によるクラスタリング結果をプロット。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロット
  plt.scatter(X_pca[assigned_class_gmm==k,0], X_pca[assigned_class_gmm==k,1], color=c, label='class: '+str(k+1))
  # セントロイドを "x"印でプロット
  plt.scatter(means[k,0], means[k,1], label='means: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

更新の様子をアニメーションでプロット

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

colors = ['r', 'b', 'g', 'c', 'm', 'y']

images = []
fig = plt.figure(figsize=(10,6))

# 更新回数の分だけ繰り返す
for step in range(len(means_history)):
  # step番目の更新時のセントロイドとクラス振り分け情報を履歴から取得する。
  assigned_class_gmm = assigned_class_gmm_history[step]
  means = means_history[step]

  # このstepのセントロイドとクラス振り分け情報をプロット
  image_tmp = []
  for k in range(K):
    c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
    # 全データのプロット
    img = plt.scatter(X_pca[assigned_class_gmm==k,0], X_pca[assigned_class_gmm==k,1], color=c, label='class: '+str(k+1))
    image_tmp.append(img)
    # セントロイドを "x"印でプロット
    img = plt.scatter(means[k,0], means[k,1], label='mean: '+str(k+1), color=c, marker='x', s=100)
    image_tmp.append(img)
  img = plt.text(-4.5, 2.5, 'Iteration: '+str(step+1), size='x-large')
  image_tmp.append(img)

  images.append(image_tmp)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=1000)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)

k-meansの結果がさらに修正され，直感通りのクラスタリング結果になるのが分かる。